In [1]:
import pandas as pd

sales_reciepts = pd.read_csv("the_sales_reciepts.csv")
customer = pd.read_csv("customer.csv")
product = pd.read_csv("product.csv")

In [2]:
Full_coffee_table = pd.merge(sales_reciepts,customer, on='customer_id')
Full_coffee_table = pd.merge(Full_coffee_table,product, on='product_id')

In [3]:
null_values = Full_coffee_table.isnull().any()
if any(null_values):
    print(null_values)


In [4]:
#make table that finds the number of orders per day
def date_to_days(date): 
    date_parts = date.split("-")
    return int(date_parts[2])

orders_by_date = Full_coffee_table.groupby('transaction_date')['transaction_id'].count().reset_index()
orders_by_date = orders_by_date.rename(columns={'transaction_date': 'Date', 'transaction_id': 'Number_of_orders'})
orders_by_date['Day'] = orders_by_date['Date'].apply(date_to_days)
orders_by_date

,Date,Number_of_orders,Day
0,2019-04-01,1657,1
1,2019-04-02,1638,2
2,2019-04-03,1681,3
3,2019-04-04,1590,4
4,2019-04-05,1634,5
5,2019-04-06,1464,6
6,2019-04-07,854,7
7,2019-04-08,889,8
8,2019-04-09,666,9
9,2019-04-10,994,10


In [5]:
!pip install dash
!pip install dash-renderer
!pip install plotly
!pip install scikit-learn

In [10]:
import dash
import dash_core_components as dcc
import dash_html_components as html
app = dash.Dash(__name__)


In [20]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.metrics import r2_score
import plotly.graph_objects as go


# Polynomial Regression Plot
xx = orders_by_date.Day
yy = orders_by_date.Number_of_orders
X_tr, X_te, y_tr, y_te = train_test_split(xx, yy, test_size=0.2)
Poly_m = PolynomialFeatures(degree=2)
X_train_p, X_test_p = Poly_m.fit_transform(X_tr.values.reshape(-1, 1)), Poly_m.fit_transform(X_te.values.reshape(-1, 1))

m = linear_model.LinearRegression()
m = m.fit(X_train_p, y_tr)
coeffi = m.coef_
intercep = m.intercept_

x_Axis2 = np.arange(1, 30, 0.1)
resp = intercep + coeffi[1] * x_Axis2 + coeffi[2] * x_Axis2 ** 2

# Calculate R-squared score
prediction = m.predict(X_test_p)
r2 = r2_score(y_te, prediction)

# Polynomial Regression Plot
polynomial_plot = dcc.Graph(
    id='polynomial-plot',
    figure={
        'data': [
            {'x': x_Axis2, 'y': resp, 'mode': 'lines', 'type': 'scatter', 'name': 'Polynomial Regression Line'},
            {'x': xx, 'y': yy, 'mode': 'markers', 'type': 'scatter', 'name': 'Number of Daily orders'}
        ],
        'layout': {
            'title': 'Polynomial Regression Plot',
            'xaxis': {'title': 'Day of the month Bought'},
            'yaxis': {'title': 'Number of Total Orders'},
            'plot_bgcolor': 'lightblue',
            'paper_bgcolor': 'black',
            'font': { 'family': 'Agency FB',
                'color': 'lightblue',
                'size': 16
            }
        }
    }
)



# Bar Plot
bar_plot = go.Figure(data=go.Bar(
    x=orders_by_date['Day'],
    y=orders_by_date['Number_of_orders'],
    marker=dict(color='lightblue'),
))

bar_plot.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
)


# Scatter Plot
scatter_plot = go.Figure(data=go.Scatter(
    x=orders_by_date['Day'],
    y=orders_by_date['Number_of_orders'],
    mode='markers',
    marker=dict(color='blue'),
))

# Update scatter plot layout
scatter_plot.update_layout(
    title='',
    xaxis=dict(title='Day of the month Bought'),
    yaxis=dict(title='Number of Total Orders'),
    plot_bgcolor='lightblue',
    paper_bgcolor='black',
    font=dict(family='Agency FB', color='lightblue', size=16)
)

# Create the Dash app
app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.Meta(
        name='viewport',
        content='width=device-width, initial-scale=1.0, maximum-scale=1.2, minimum-scale=0.5'
    ),
    html.Div(
        style={'background-color': 'black'},
        children=[
            html.Div(
                children=[
                    html.H1('A diagram showing the number of total orders per day', style={'color': 'lightblue', 'font-family': 'Agency FB','text-align': 'center'}),
                    dcc.Graph(
                        id='scatter-plot',
                        figure=scatter_plot
                    )
                ],
                style={'width': '75%', 'margin': '0 auto'}
            ),
            html.Div(
                children=[
                    html.H1('Bar Plot of Number of Orders', style={'color': 'lightblue', 'font-family': 'Agency FB', 'text-align': 'center'}),
                    dcc.Graph(
                        id='bar-plot',
                        figure=bar_plot,
                    )
                ],
                style={'width': '50%', 'float': 'right', 'background-color': 'black', 'margin': '0 auto'}
            ),
             html.Div(
                children=[
                    html.H1('Polynomial Regression Plot', style={'color': 'lightblue', 'font-family': 'Agency FB', 'text-align': 'center'}),
                    polynomial_plot,
                    
                ],
                style={'width': '50%', 'float': 'left', 'margin': '0 auto', 'background-color': 'black'}
            ),
            html.Div(
                children=[
                    html.H2(f'Correlation value between orders and the day: {r2:.4f}',
                            style={'color': 'lightblue', 'font-family': 'Agency FB', 'font-size': 12.7, 'text-align': 'left'})
                ],
                style={'width': '100%', 'margin': '0 auto', 'background-color': 'black'}
            )
           
        ]
    )
])

In [21]:
if __name__ == '__main__':
     app.run_server(host='localhost',port=8005)

Dash is running on http://localhost:8005/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:8005
Press CTRL+C to quit
127.0.0.1 - - [19/May/2023 15:29:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 15:29:40] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 15:29:41] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 15:29:41] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [19/May/2023 15:29:41] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [19/May/2023 16:59:23] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 16:59:23] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 16:59:24] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [19/May/2023 16:59:24] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [19/May/2023 16:59:24] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 16:59:24] "GET /_dash-dependencies HTTP/1.1" 200 -
127